In [11]:
from inference_on_dataset import NEGATIVE_PROMPTS, get_pipe, load_ckpt
from omegaconf import OmegaConf
import json
import numpy as np
import torch

In [ ]:
version = 'sdxl'
config = 'configs/layout_sdxl.yaml'
config = OmegaConf.load(config)
pipe, _, unet = get_pipe(version, config)

In [6]:
load_ckpt('rich_context_sdxl_e580.pt', pipe)

In [16]:
height = width = 512
GUIDANCE_SCALE = 5.0
caption = '' # You can leave it blank or give it a global caption
object_file = 'demos/100002558.json'
with open(object_file, 'r') as f:
    objects = json.load(f)
    bboxes = np.array(objects['bboxes'])
    labels = objects['labels']

In [ ]:
with torch.no_grad(), torch.cuda.amp.autocast(dtype=torch.float16):
    generated = pipe(
        prompt=caption,
        negative_prompt=NEGATIVE_PROMPTS,
        height=height, width=width,
        boxes=bboxes,
        labels=labels,
        scheduled_sampling_beta=1,
        num_inference_steps=50,
        guidance_scale=GUIDANCE_SCALE
    )

In [ ]:
generated[0][0]